In [139]:
# import the necessary Libraries 
import sys
import requests
import json

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors


import io
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

from sklearn.cluster import KMeans

In [140]:
#We get the information of postal codes for the major districts in Tokyo from the following website

url = "http://japanzipcodes.blogspot.com/2013/08/the-major-zip-codes-of-tokyo-japan.html"

url_u = requests.get(url).text

soup = BeautifulSoup(url_u, 'html.parser')

#We need to extract the information and store it in two lists: "district" and "postal_code" 

zips = soup.find(id="post-body-1523308068074273552")


tokyo_list = list(zips)



district = []
postal_code = []


for i in range(0,108,2):
    index = tokyo_list[i]
    if i not in (74,76,78):
        district_index = index.split(',')[0].split('      ')[1].split(' ')[0]
        postal_code_index = index.split(',')[2].split('is ')[1].split(' ')[0]    
        #append items
        district.append(district_index)
        postal_code.append(postal_code_index)    
        #three items have different parsing
    else: 
        district_index = index.split(',')[0].split('    ')[1].split(' ')[0]
        postal_code_index = index.split(',')[3].split('is ')[1].split(' ')[0]
        #append items
        district.append(district_index)
        postal_code.append(postal_code_index)

In [141]:
#We build a dataframe including the districts and the corresponding postal_codes
tokyo = {'District':district,'Postal_Code':postal_code}
tokyo_df = pd.DataFrame(tokyo)
#Adachiku is better known as Adachi. So we change that to not get a problem later in geolocating
#Same for a few other
tokyo_df['District'][0] = "Adachi"
tokyo_df['District'][4] = "Bunkyo"
tokyo_df['District'][5] = "Chiyoda"
tokyo_df['District'][18] = "Itabashi"
tokyo_df['District'][19] = "Katsushika"
tokyo_df['District'][36] = "Niijima"
tokyo_df['District'][39] = "Okutama"
tokyo_df['District'][43] = "Oshima"
tokyo_df['District'][46] = "Shibuya"
tokyo_df['District'][47] = "Shinagawa"
tokyo_df['District'][48] = "Shinjuku"

In [142]:
tokyo_df

,District,Postal_Code
0,Adachi,120-0000
1,Akiruno,190-0100
2,Akishima,196-0000
3,Arakawaku,116-0000
4,Bunkyo,112-0000
5,Chiyoda,100-0000
6,Chofu,182-0000
7,Chuoku,103-0000
8,Edogawaku,132-0000
9,Fuchu,183-0000


In [143]:
districts = tokyo_df['District']

lat = []
lon = []

from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="my_user_agent")
city ="Tokyo"
country ="Japan"

for i in districts:
    print(i)
    loc = geolocator.geocode(i +','+"Tokyo"+','+ "Japan")
    lat.append(loc.latitude)
    lon.append(loc.longitude)


Adachi
Akiruno
Akishima
Arakawaku
Bunkyo
Chiyoda
Chofu
Chuoku
Edogawaku
Fuchu
Fussa
Hachioji
Hamura
Higashikurume
Higashimurayama
Higashiyamato
Hino
Inagi
Itabashi
Katsushika
Kitaku
Kiyose
Kodaira
Koganei
Kokubunji
Komae
Kotoku
Kunitachi
Machida
Meguroku
Minatoku
Mitaka
Musashimurayama
Musashino
Nakanoku
Nerimaku
Niijima
Hinodemachi
Hinoharamura
Okutama
Nishitokyo
Ogasawaramura
Ome
Oshima
Otaku
Setagayaku
Shibuya
Shinagawa
Shinjuku
Suginamiku
Sumidaku
Tachikawa
Taitoku
Tama


In [145]:
#add the coordinates to the existing data frame
tokyo_df['Latitude'] = lat
tokyo_df['Longitude'] = lon

In [146]:
tokyo_df

,District,Postal_Code,Latitude,Longitude
0,Adachi,120-0000,35.783703,139.795319
1,Akiruno,190-0100,35.731042,139.217028
2,Akishima,196-0000,35.702480,139.350065
3,Arakawaku,116-0000,35.737529,139.781310
4,Bunkyo,112-0000,35.718810,139.744732
5,Chiyoda,100-0000,35.685403,139.752738
6,Chofu,182-0000,35.660036,139.554815
7,Chuoku,103-0000,34.545288,135.840885
8,Edogawaku,132-0000,35.706653,139.868312
9,Fuchu,183-0000,35.672219,139.480009


In [147]:
#coordinates of Tokyo

address = 'Tokyo, Japan'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Tokyo are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Tokyo are 35.6828387, 139.7594549.


In [149]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [150]:
# create map of Tokyo
map_manhattan = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers off the districts
for lat, lng, label in zip(tokyo_df['Latitude'], tokyo_df['Longitude'], tokyo_df['District']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_manhattan)  
    
map_manhattan

In [151]:
CLIENT_ID = '2ZX4HJMJUV2EXPLKE3JGJPEEJGZ2WSBQA4UL0DHI1ZWIEX2U' # your Foursquare ID
CLIENT_SECRET = 'EY145003SSWRH5H321SPTJEVX33D2SRUXK2M01Y331FCTTJX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 2ZX4HJMJUV2EXPLKE3JGJPEEJGZ2WSBQA4UL0DHI1ZWIEX2U
CLIENT_SECRET:EY145003SSWRH5H321SPTJEVX33D2SRUXK2M01Y331FCTTJX


In [153]:
#we use the function getNearbyVenues from the tutorial and modify it so that we get venues that are found via "video games"
#therefore we include the query 'video games'
#radius = 500 seems like a good choice, so we leave the standard value from the tutorial

def getNearbyVenues(names, latitudes, longitudes, radius=500, query = 'video games'):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT,
            query)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [154]:
tokyo_vgs = getNearbyVenues(names=tokyo_df['District'],
                                   latitudes=tokyo_df['Latitude'],
                                   longitudes=tokyo_df['Longitude']
                                  )

Adachi
Akiruno
Akishima
Arakawaku
Bunkyo
Chiyoda
Chofu
Chuoku
Edogawaku
Fuchu
Fussa
Hachioji
Hamura
Higashikurume
Higashimurayama
Higashiyamato
Hino
Inagi
Itabashi
Katsushika
Kitaku
Kiyose
Kodaira
Koganei
Kokubunji
Komae
Kotoku
Kunitachi
Machida
Meguroku
Minatoku
Mitaka
Musashimurayama
Musashino
Nakanoku
Nerimaku
Niijima
Hinodemachi
Hinoharamura
Okutama
Nishitokyo
Ogasawaramura
Ome
Oshima
Otaku
Setagayaku
Shibuya
Shinagawa
Shinjuku
Suginamiku
Sumidaku
Tachikawa
Taitoku
Tama


In [155]:
tokyo_vgs

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Adachi,35.783703,139.795319,7-Eleven (セブンイレブン 足立島根店),35.782408,139.797570,Convenience Store
1,Adachi,35.783703,139.795319,ウエルシア薬局 足立島根店,35.780319,139.797193,Drugstore
2,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根二丁目店),35.782771,139.799411,Convenience Store
3,Adachi,35.783703,139.795319,COCO'S (ココス),35.782871,139.797551,Japanese Family Restaurant
4,Adachi,35.783703,139.795319,FamilyMart (ファミリーマート 島根四丁目店),35.782487,139.792864,Convenience Store
...,...,...,...,...,...,...,...
1301,Tama,35.637188,139.443503,Dining和桜,35.640556,139.444600,Japanese Restaurant
1302,Tama,35.637188,139.443503,諏訪下橋バス停,35.636195,139.447745,Bus Stop
1303,Tama,35.637188,139.443503,Sakuragaoka Rotary (桜ヶ丘ロータリー (地球屋前ロータリー)),35.640814,139.444469,Road
1304,Tama,35.637188,139.443503,原峰公園,35.640998,139.445108,Park
